In [207]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import calendar
from utils import *
import quantstats as qs
import scipy
from sklearn.linear_model import LinearRegression
from tqdm import tqdm


In [83]:
# Get the crypto fund data from Vision Track
data = getDataVisionTrack("./Data/Vision track fund data/VisionTrack by Vision Hill Group.html")

# Getting the monthly risk free rates
rf = loadRiskFreeRate("./Data/United States 5-Year Bond Yield Historical Data monthly long term.csv").set_index("Date")
monthlyRF = rf.groupby(pd.Grouper(freq='Y')).apply(lambda x: np.power(1+x.mean(),1/12)-1)
monthlyRF


,Price
Date,
2013-12-31,0.001223
2014-12-31,0.001342
2015-12-31,0.001238
2016-12-31,0.001096
2017-12-31,0.001589
2018-12-31,0.002251
2019-12-31,0.001566
2020-12-31,0.000379
2021-12-31,0.000738


In [209]:
# Objective function: Sharpe ratio
# Get a list of all the months that we have trading data for
windowSize = 2

months = []
dateEnd = datetime.date(2023, 12, 1)    
current = datetime.date(2013, 7, 1)    
while current <= dateEnd:
    months.append(current)
    current += relativedelta(months=1)
    
months = months[::-1]
months = pd.to_datetime(months)

dfObjectiveSharpe = pd.DataFrame(np.nan, columns=data["Fund Name"], index = months)
dfObjectiveSharpe.index = pd.to_datetime(dfObjectiveSharpe.index)

for fund in tqdm(data.iterrows(), total=data.shape[0]):

    def tempCalcSharpe(rt, rf):
        rf = rf[rt.index.max()<=rf.index].iloc[0,0]
        return qs.stats.sharpe(rt, rf = rf, annualize=False)

    fundName = fund[1][0]
    returns = fund[1][3:].str.replace("%","").astype(float).div(100)
    returns.index = months
    sharpeReturns = returns.dropna().rolling(windowSize, min_periods=windowSize).apply(lambda x: tempCalcSharpe(x, monthlyRF) )
    
    dfObjectiveSharpe.loc[
        (sharpeReturns.index.min() < dfObjectiveSharpe.index) & (dfObjectiveSharpe.index < sharpeReturns.index.max()) ,
        fundName] = sharpeReturns


100%|██████████| 294/294 [02:41<00:00,  1.82it/s]


In [282]:
# Objective function: max drawdown
# Get a list of all the months that we have trading data for
windowSize = 5

months = []
dateEnd = datetime.date(2023, 12, 1)    
current = datetime.date(2013, 7, 1)    
while current <= dateEnd:
    months.append(current)
    current += relativedelta(months=1)
    
months = months[::-1]
months = pd.to_datetime(months)

dfObjectiveMDD = pd.DataFrame(np.nan, columns=data["Fund Name"], index = months)
dfObjectiveMDD.index = pd.to_datetime(dfObjectiveMDD.index)

for fund in tqdm(data.iterrows(), total=data.shape[0]):

    def tempCalcMDD(rt):
        rt = rt.sort_index(ascending=True)  
        rt = (rt.dropna()+1).cumprod()
        Roll_Max = rt.cummax()
        Daily_Drawdown = rt/Roll_Max - 1.0
        Max_Drawdown = Daily_Drawdown.cummin()

        return Max_Drawdown[-1]

    fundName = fund[1][0]
    returns = fund[1][3:].str.replace("%","").astype(float).div(100)
    returns.index = months
    MDDReturns = returns.dropna().rolling(windowSize, min_periods=windowSize).apply(lambda x: tempCalcMDD(x) )
    
    dfObjectiveMDD.loc[
        (MDDReturns.index.min() < dfObjectiveMDD.index) & (dfObjectiveMDD.index < MDDReturns.index.max()) ,
        fundName] = MDDReturns

100%|██████████| 294/294 [00:06<00:00, 48.00it/s]


In [288]:
# Objective function: return
# Get a list of all the months that we have trading data for
windowSize = 5

months = []
dateEnd = datetime.date(2023, 12, 1)    
current = datetime.date(2013, 7, 1)    
while current <= dateEnd:
    months.append(current)
    current += relativedelta(months=1)
    
months = months[::-1]
months = pd.to_datetime(months)

dfObjectivecumReturn = pd.DataFrame(np.nan, columns=data["Fund Name"], index = months)
dfObjectivecumReturn.index = pd.to_datetime(dfObjectivecumReturn.index)

for fund in tqdm(data.iterrows(), total=data.shape[0]):

    def tempCalccumReturn(rt):
        rt = rt.sort_index(ascending=True)  
        rt = (rt.dropna()+1).cumprod()

        return rt[-1]

    fundName = fund[1][0]
    returns = fund[1][3:].str.replace("%","").astype(float).div(100)
    returns.index = months
    cumReturnReturns = returns.dropna().rolling(windowSize, min_periods=windowSize).apply(lambda x: tempCalccumReturn(x) )
    
    dfObjectivecumReturn.loc[
        (cumReturnReturns.index.min() < dfObjectivecumReturn.index) & (dfObjectivecumReturn.index < cumReturnReturns.index.max()) ,
        fundName] = cumReturnReturns

100%|██████████| 294/294 [00:04<00:00, 68.76it/s]


In [290]:
dfObjectiveSharpe.to_excel("objective-sharpe.xlsx")
dfObjectiveMDD.to_excel("objective-MDD.xlsx")